In [1]:
import pandas as pd
import requests 
import json
from config import gkey
import numpy as np
from sqlalchemy import create_engine

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_the_United_States_National_Park_System_official_units'

In [3]:
tables = pd.read_html(url)

In [4]:
raw_df = tables[1]
df = raw_df.drop_duplicates(subset='Name', keep="last")
print(len(df))

421


In [5]:
states_that_matter = ["Washington", "Oregon", "California", "Nevada", "Idaho", "Montana", "Wyoming", "Utah",
                      "Arizona", "New Mexico", "Colorado", "North Dakota", "South Dakota", "Nebraska", "Kansas", 
                      "Oklahoma", "Texas"]

In [6]:
#filtering out states east of the Mississippi
for name in df["Location"]:
    if name not in states_that_matter: 
        df.drop(df.loc[df['Location']==name].index, inplace=True)
print(len(df))


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


156


In [7]:
us_parks = df.reset_index(drop=True)
us_parks.count()

Name        156
Location    156
dtype: int64

In [ ]:
##DO NOT RUN NOW
raw_national_parks = us_parks.loc[us_parks['Name'].str.endswith('National Park')]
national_parks = raw_national_parks.reset_index(drop=True)

raw_national_monuments = us_parks.loc[us_parks['Name'].str.endswith('National Monument')]
national_monuments = raw_national_monuments.reset_index(drop=True)

raw_recreation_areas = us_parks.loc[us_parks['Name'].str.endswith('Recreation Area')]
recreation_areas = raw_recreation_areas.reset_index(drop=True)

In [8]:
us_parks["type"] = ""

In [9]:
us_parks.loc[us_parks.Name.str.endswith('National Park'), 'type'] = 'National Park'
us_parks.loc[us_parks.Name.str.endswith('National Monument'), 'type'] = 'National Monument'
us_parks.loc[us_parks.Name.str.endswith('Recreation Area'), 'type'] = 'Recreation Area'

In [10]:
## dropping everything that isnt a National Park, Monument or Recreation Area

i = us_parks[(us_parks.type == '')].index
us_parks.drop(i, inplace = True)

In [11]:
#had to remove this park because it will break the API call loop, island can't be driven to
us_parks.drop(us_parks.loc[us_parks['Name'] == "Channel Islands National Park"].index, inplace = True)
us_parks.drop(us_parks.loc[us_parks['Name'] == "Rainbow Bridge National Monument"].index, inplace = True)
us_parks.drop(us_parks.loc[us_parks['Name'] == "Lake Chelan National Recreation Area"].index, inplace = True)


In [12]:
us_parks.rename(columns = {"Name" : "name", "Location" : 'location'}, inplace = 'True')

In [ ]:
us_parks.head(100)

cities = {dallas, austin, houstin}

In [13]:
lat_list = []
lng_list = []

for park in us_parks['name']:
    target_park = park
    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {"input": target_park,"key": gkey, "inputtype" : "textquery", "fields" : "geometry"}
    response = requests.get(base_url, params=params)
    trip = response.json()
    
    lat = trip["candidates"][0]["geometry"]["location"]["lat"]
    lng = trip["candidates"][0]["geometry"]["location"]["lng"]
    
    lat_list.append(lat)
    lng_list.append(lng)
    
us_parks.insert(3, 'lat', lat_list)
us_parks.insert(4, 'lng', lng_list)   


In [ ]:
data = pd.read_sql("SELECT * FROM Census_Data", conn)

In [14]:
#all working

##dont run on first iteration

target_city = "Los Angeles"

distances = []
durations = []
for park in us_parks['name']:
    #target_city = "Portland"
    target_park = park

    base_url = "https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial"

    params = {"origins": target_city, "destinations": target_park, "key": gkey}

    response = requests.get(base_url, params=params)
    try:
        trip = response.json()

        distance = trip["rows"][0]["elements"][0]["distance"]["text"]
        duration = trip["rows"][0]["elements"][0]["duration"]["value"]

        hours = round((duration/3600),2)

        if "." in distance:
            new_distance = float(distance.strip('mi').replace(',', ''))
        else:
            new_distance = int(distance.strip('mi').replace(',', ''))

        distances.append(new_distance)
        durations.append(hours)
    except:
        distances.append(0)
        durations.append(0)


us_parks.insert(5, 'distance', distances)
us_parks.insert(6, 'duration', durations)

# add this shit to the database so I dont have to query Google every time 

In [15]:
us_parks.head()

,name,location,type,lat,lng,distance,duration
1,Canyon De Chelly National Monument,Arizona,National Monument,36.133611,-109.469444,633.0,9.75
2,Casa Grande Ruins National Monument,Arizona,National Monument,32.995459,-111.535528,426.0,6.35
3,Chiricahua National Monument,Arizona,National Monument,32.013571,-109.342295,601.0,8.92
6,Glen Canyon National Recreation Area,Utah,Recreation Area,37.386726,-110.842426,525.0,7.94
7,Grand Canyon National Park,Arizona,National Park,36.106926,-112.112948,486.0,7.36


In [16]:
rds_connection_string = "postgres:postgres@localhost:5432/Parks"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [17]:
def remove(string): 
    return string.replace(" ", "")

In [18]:
new_name = remove(target_city)
new_name

'LosAngeles'

In [19]:
#us_parks.to_sql(target_city, engine)
us_parks.to_sql(name=new_name.lower(), con=engine, if_exists='replace', index=False)

In [22]:
data = engine.execute(f'alter table {new_name} add primary key (name)')

In [ ]:
close_index = int(input("What is considered close to you in hours?"))
near_index = int(input("What is considered near to you in hours?"))
far_index = int(input("What is considered far to you in hours?"))

far_national_parks = national_parks[national_parks['Duration'] <= far_index]

medium_national_parks = national_parks[(national_parks['Duration'] >= close_index) & (national_parks['Duration'] < far_index)]

close_national_parks = national_parks[national_parks['Duration'] < close_index]

print("Here is what you could reach within those criteria")
#print(len(medium_natinoal_parks))
#print(len(close_national_parks))
#print(len(far_natinoal_parks))
close_national_parks

In [ ]:
medium_national_parks

In [ ]:
far_national_parks

In [ ]:
close_index = int(input("What is considered close to you in hours?"))
near_index = int(input("What is considered near to you in hours?"))
far_index = int(input("What is considered far to you in hours?"))

far_national_parks = national_parks[national_parks['Duration'] >= far_index]

medium_national_parks = national_parks[(national_parks['Duration'] >= close_index) & (national_parks['Duration'] < far_index)]

close_national_parks = national_parks[national_parks['Duration'] < near_index]

print("Here is what you could reach within those criteria")
#print(len(medium_natinoal_parks))
#print(len(close_national_parks))
#print(len(far_natinoal_parks))
close_national_parks

In [ ]:
##for scalability, when user inputs a new city, it uses the Google API to find all distances. After a city has been queried onve, the dabase is accessed instead for speed purposes

#when you click a park, it takes you to a wiki page